In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from xgboost import XGBClassifier
import joblib

In [2]:
df=pd.read_csv(r"C:\Users\punee\OneDrive\Desktop\Transaction Fraud Model\rhmlmodel\Data\Processed\Transaction_fraud_data_processed.csv")
df

,step,type,amount,Sender,oldbalanceSender,newbalanceSender,Receiver,oldbalanceReceiver,newbalanceReceiver,isFraud,isFlaggedFraud,balance_change_sender,balance_change_receiver,mismatch_flag,prev_transactions_sender,prev_receives_receiver
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0,9839.64,0.00,True,0,0
1,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,181.00,0.00,True,0,0
2,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0,181.00,21182.00,True,0,0
3,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0,11668.14,0.00,True,0,0
4,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.00,0.00,0,0,7817.71,0.00,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,1258818.82,C1436118706,1258818.82,0.00,C1240760502,503464.50,1762283.33,1,0,1258818.82,-1258818.83,True,0,5
6362616,743,TRANSFER,339682.13,C2013999242,339682.13,0.00,C1850423904,0.00,0.00,1,0,339682.13,0.00,True,0,0
6362617,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0,339682.13,-339682.13,False,0,1
6362618,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0,6311409.28,0.00,True,0,0


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score


df['type'] = LabelEncoder().fit_transform(df['type'])

# Drop unnecessary columns
columns_to_drop = ['isFraud', 'oldbalanceSender', 'newbalanceSender', 'oldbalanceReceiver', 'newbalanceReceiver', 'Sender', 'Receiver']
X = df.drop(columns=columns_to_drop)
y = df['isFraud']

# Separate majority and minority classes
df_majority = df[df['isFraud'] == 0]
df_minority = df[df['isFraud'] == 1]

# Number of iterations
num_iterations = 10

# Models to evaluate
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    "LightGBM": LGBMClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    "CatBoost": CatBoostClassifier(iterations=200, learning_rate=0.1, depth=6, random_state=42, verbose=0),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(probability=True, random_state=42)
}

# Dictionary to store results
results = {model: {"accuracy": [], "roc_auc": []} for model in models}

# Run multiple iterations
for i in range(num_iterations):
    print(f"\nIteration {i+1}/{num_iterations}")

    # Randomly sample non-fraud cases
    df_majority_sampled = df_majority.sample(n=len(df_minority), random_state=i)
    df_balanced = pd.concat([df_majority_sampled, df_minority])

    # Prepare X and y
    X_balanced = df_balanced.drop(columns=columns_to_drop)
    y_balanced = df_balanced['isFraud']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced)

    # Feature Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train and evaluate each model
    for model_name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

        # Compute metrics
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)

        results[model_name]["accuracy"].append(accuracy)
        results[model_name]["roc_auc"].append(roc_auc)

        print(f"{model_name}: Accuracy={accuracy:.4f}, ROC-AUC={roc_auc:.4f}")

# Compute average results
print("\nFinal Results:")
for model_name in models:
    avg_acc = np.mean(results[model_name]["accuracy"])
    std_acc = np.std(results[model_name]["accuracy"])
    avg_roc_auc = np.mean(results[model_name]["roc_auc"])
    std_roc_auc = np.std(results[model_name]["roc_auc"])

    print(f"{model_name}: Avg Accuracy={avg_acc:.4f} ± {std_acc:.4f}, Avg ROC-AUC={avg_roc_auc:.4f} ± {std_roc_auc:.4f}")



Iteration 1/10
Logistic Regression: Accuracy=0.9029, ROC-AUC=0.9648
Decision Tree: Accuracy=0.9665, ROC-AUC=0.9665
Random Forest: Accuracy=0.9741, ROC-AUC=0.9973
XGBoost: Accuracy=0.9738, ROC-AUC=0.9980
[LightGBM] [Info] Number of positive: 6570, number of negative: 6570
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000806 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1087
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM: Accuracy=0.9738, ROC-AUC=0.9979
CatBoost: Accuracy=0.9769, ROC-AUC=0.9979
KNN: Accuracy=0.9373, ROC-AUC=0.9760
SVM: Accuracy=0.9394, ROC-AUC=0.9841

Iteration 2/10
Logistic Regression: Accuracy=0.9041, ROC-AUC=0.9661
Decision Tree: Accuracy=0.9610, ROC-AUC=0.9610
Random Forest: Accuracy=0.9702, ROC-AUC=0.9972
XGBoost: Accuracy=0.9717, ROC-AUC=0.9978
[LightGBM] [Info] Number of positive: 6570, number of negative: 6570
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1086
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


CatBoost: Accuracy=0.9723, ROC-AUC=0.9976
KNN: Accuracy=0.9413, ROC-AUC=0.9767
SVM: Accuracy=0.9367, ROC-AUC=0.9829

Iteration 3/10
Logistic Regression: Accuracy=0.9032, ROC-AUC=0.9664
Decision Tree: Accuracy=0.9644, ROC-AUC=0.9644
Random Forest: Accuracy=0.9747, ROC-AUC=0.9971
XGBoost: Accuracy=0.9784, ROC-AUC=0.9979
[LightGBM] [Info] Number of positive: 6570, number of negative: 6570
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000454 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1086
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

...

CatBoost: Accuracy=0.9720, ROC-AUC=0.9972
KNN: Accuracy=0.9385, ROC-AUC=0.9770
SVM: Accuracy=0.9376, ROC-AUC=0.9815

Iteration 9/10
Logistic Regression: Accuracy=0.9081, ROC-AUC=0.9670
Decision Tree: Accuracy=0.9620, ROC-AUC=0.9620
Random Forest: Accuracy=0.9735, ROC-AUC=0.9970
XGBoost: Accuracy=0.9763, ROC-AUC=0.9979
[LightGBM] [Info] Number of positive: 6570, number of negative: 6570
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000464 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1085
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM: Accuracy=0.9729, ROC-AUC=0.9976
CatBoost: Accuracy=0.9757, ROC-AUC=0.9975
KNN: Accuracy=0.9434, ROC-AUC=0.9763
SVM: Accuracy=0.9400, ROC-AUC=0.9847

Iteration 10/10
Logistic Regression: Accuracy=0.9038, ROC-AUC=0.9665
Decision Tree: Accuracy=0.9550, ROC-AUC=0.9550
Random Forest: Accuracy=0.9732, ROC-AUC=0.9975
XGBoost: Accuracy=0.9775, ROC-AUC=0.9983
[LightGBM] [Info] Number of positive: 6570, number of negative: 6570
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000429 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1084
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\punee\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


CatBoost: Accuracy=0.9772, ROC-AUC=0.9982
KNN: Accuracy=0.9407, ROC-AUC=0.9775
SVM: Accuracy=0.9376, ROC-AUC=0.9850

Final Results:
Logistic Regression: Avg Accuracy=0.9030 ± 0.0024, Avg ROC-AUC=0.9654 ± 0.0010
Decision Tree: Avg Accuracy=0.9617 ± 0.0030, Avg ROC-AUC=0.9617 ± 0.0030
Random Forest: Avg Accuracy=0.9719 ± 0.0018, Avg ROC-AUC=0.9970 ± 0.0003
XGBoost: Avg Accuracy=0.9752 ± 0.0025, Avg ROC-AUC=0.9978 ± 0.0003
LightGBM: Avg Accuracy=0.9743 ± 0.0017, Avg ROC-AUC=0.9977 ± 0.0003
CatBoost: Avg Accuracy=0.9748 ± 0.0018, Avg ROC-AUC=0.9976 ± 0.0003
KNN: Avg Accuracy=0.9404 ± 0.0032, Avg ROC-AUC=0.9772 ± 0.0013
SVM: Avg Accuracy=0.9381 ± 0.0014, Avg ROC-AUC=0.9833 ± 0.0011


In [4]:
import joblib

# Encode categorical variable
df['type'] = LabelEncoder().fit_transform(df['type'])

# Drop unnecessary columns
columns_to_drop = ['isFraud', 'oldbalanceSender', 'newbalanceSender', 
                   'oldbalanceReceiver', 'newbalanceReceiver', 'Sender', 'Receiver']
X = df.drop(columns=columns_to_drop)
y = df['isFraud']

# Separate majority and minority classes
df_majority = df[df['isFraud'] == 0]
df_minority = df[df['isFraud'] == 1]

# Number of iterations
num_iterations = 10

# Store results
accuracy_list = []
roc_auc_list = []
best_model = None
best_roc_auc = 0

for i in range(num_iterations):
    print(f"\nIteration {i+1}/{num_iterations}")

    # Randomly sample non-fraud cases
    df_majority_sampled = df_majority.sample(n=len(df_minority), random_state=i)
    df_balanced = pd.concat([df_majority_sampled, df_minority])

    # Prepare X and y
    X_balanced = df_balanced.drop(columns=columns_to_drop)
    y_balanced = df_balanced['isFraud']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, 
                                                        random_state=42, stratify=y_balanced)

    # Feature Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train XGBoost Model
    model = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    accuracy_list.append(accuracy)
    roc_auc_list.append(roc_auc)

    # Save best model based on ROC-AUC
    if roc_auc > best_roc_auc:
        best_roc_auc = roc_auc
        best_model = model
        best_feature_names = list(X_train.columns)  # Save feature names

    print(f"Iteration {i+1} - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")

# Save the best model, scaler, and feature names
if best_model:
    joblib.dump(best_model, "xgboost_fraud_model.pkl")
    joblib.dump(scaler, "scaler.pkl")
    joblib.dump(best_feature_names, "feature_names.pkl")  # Save feature names

    print(f"\nBest XGBoost model saved as 'xgboost_fraud_model.pkl'.")
    print("Scaler saved as 'scaler.pkl'.")
    print("Feature names saved as 'feature_names.pkl'.")



Iteration 1/10
Iteration 1 - Accuracy: 0.9738, ROC-AUC: 0.9980

Iteration 2/10
Iteration 2 - Accuracy: 0.9717, ROC-AUC: 0.9978

Iteration 3/10
Iteration 3 - Accuracy: 0.9784, ROC-AUC: 0.9979

Iteration 4/10
Iteration 4 - Accuracy: 0.9738, ROC-AUC: 0.9976

Iteration 5/10
Iteration 5 - Accuracy: 0.9784, ROC-AUC: 0.9981

Iteration 6/10
Iteration 6 - Accuracy: 0.9757, ROC-AUC: 0.9974

Iteration 7/10
Iteration 7 - Accuracy: 0.9757, ROC-AUC: 0.9975

Iteration 8/10
Iteration 8 - Accuracy: 0.9711, ROC-AUC: 0.9972

Iteration 9/10
Iteration 9 - Accuracy: 0.9763, ROC-AUC: 0.9979

Iteration 10/10
Iteration 10 - Accuracy: 0.9775, ROC-AUC: 0.9983

Best XGBoost model saved as 'xgboost_fraud_model.pkl'.
Scaler saved as 'scaler.pkl'.
Feature names saved as 'feature_names.pkl'.


In [27]:
import joblib
import pandas as pd

# Load trained model, scaler, and feature names
model = joblib.load("xgboost_fraud_model.pkl")
scaler = joblib.load("scaler.pkl")
feature_names = joblib.load("feature_names.pkl")  # Load feature names used during training

# Transaction type mapping
type_mapping = {"PAYMENT": 0, "TRANSFER": 1, "CASH_OUT": 2, "DEBIT": 3, "CASH_IN": 4}

def test_fraud_model(transaction_data):
    """
    Function to test the trained XGBoost fraud detection model.
    Args:
        transaction_data (dict): Transaction details.
    """

    # Convert input to DataFrame
    df = pd.DataFrame([transaction_data])

    # Convert categorical feature
    df["type"] = df["type"].map(type_mapping)

    # Compute missing derived features
    df["balance_change_sender"] = df["newbalanceSender"] - df["oldbalanceSender"]
    df["balance_change_receiver"] = df["newbalanceReceiver"] - df["oldbalanceReceiver"]
    df["mismatch_flag"] = (df["balance_change_sender"] != -1*df["balance_change_receiver"])

    # Ensure correct feature order and missing features are handled
    missing_features = set(feature_names) - set(df.columns)
    if missing_features:
        raise ValueError(f"Missing features in input: {missing_features}")

    X_test = df[feature_names]  # Ensure correct feature order

    # Scale the features
    X_test_scaled = scaler.transform(X_test)

    # Make prediction
    prediction = model.predict(X_test_scaled)[0]
    probability = model.predict_proba(X_test_scaled)[0][1]

    # Print result
    print("\n--- Prediction Result ---")
    print(f"Fraud Prediction: {'Fraudulent' if prediction == 1 else 'Not Fraudulent'}")
    print(f"Fraud Probability: {probability:.4f}\n")

# Example Test
test_transaction = {
    "step": 100,
    "type": "TRANSFER",
    "amount": 50000.0,
    "oldbalanceSender": 100000.0,
    "newbalanceSender": 50000.0,
    "oldbalanceReceiver": 20000.0,
    "newbalanceReceiver": 70000.0,
    "isFlaggedFraud": 0,
    "prev_transactions_sender": 5,
    "prev_receives_receiver": 3
}

test_fraud_model(test_transaction)



--- Prediction Result ---
Fraud Prediction: Not Fraudulent
Fraud Probability: 0.0065

